In [1]:
%logstop
%logstart -rtq ~/.logs/dw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [2]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [3]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [4]:
import pandas as pd
import numpy as np
import gzip
import os
os.listdir()

['.ipynb_checkpoints',
 'in.ipynb',
 'dw-data',
 'dw.ipynb',
 'pw.ipynb',
 'pw-data']

In [5]:
import gzip
with gzip.open('./dw-data/201701scripts_sample.csv.gz', 'rb') as f:
    file_content = f.read()
    


In [6]:
# load the 2017 data

scripts = pd.read_csv('./dw-data/201701scripts_sample.csv.gz', compression='gzip')
scripts.head()

In [7]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = pd.read_csv('./dw-data/practices.csv.gz', compression='gzip')
practices.columns = col_names
practices.head()

In [8]:
chem = pd.read_csv('./dw-data/chem.csv.gz', compression='gzip')
chem.head()


Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [9]:
clist = ['items', 'quantity', 'nic', 'act_cost']
scripts[clist[1]].describe()
scripts['items']

data = scripts['items']
ctotal = sum(data)
print(ctotal)


8888304


In [10]:
scripts[clist[1]].describe()

count    973193.000000
mean        741.329835
std        3665.426958
min           0.000000
25%          28.000000
50%         100.000000
75%         350.000000
max      577720.000000
Name: quantity, dtype: float64

In [11]:
scripts['items'].describe()

newlist= []
summary_stats = []
for item in clist:
    data = scripts[item]
    ctotal = sum(data)
    #ctotal = scripts[item].describe()[0]
    cmean = scripts[item].describe()[1]
    cstd = scripts[item].describe()[2]
    cq25 = scripts[item].describe()[4]
    cmedian = scripts[item].describe()[5]
    cq75 = scripts[item].describe()[6]
    innertupple = tuple([ctotal, cmean, cstd, cq25, cmedian, cq75 ])
    newlist = tuple([item, innertupple ])
    summary_stats.append(newlist)
    
summary_stats
#for item im scripts.

[('items', (8888304, 9.133135976111625, 29.204198282803603, 1.0, 2.0, 6.0)),
 ('quantity',
  (721457006, 741.3298348837282, 3665.426958467915, 28.0, 100.0, 350.0)),
 ('nic',
  (71100424.84000827, 73.05891517920908, 188.070256906825, 7.8, 22.64, 65.0)),
 ('act_cost',
  (66164096.11999956,
   67.98661326170655,
   174.40170332301963,
   7.33,
   21.22,
   60.67))]

In [12]:
#summary_stats = [('items', (0,) * 6), ('quantity', (0,) * 6), ('nic', (0,) * 6), ('act_cost', (0,) * 6)]

In [13]:
grader.score.dw__summary_statistics(summary_stats)

Your score:  1.0


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [14]:
by_bnfname = scripts.groupby('bnf_name')['items'].sum()
by_bnfname.head()
by_bnfname.nlargest(1)
most = []
for k,v in by_bnfname.nlargest(1).items():
    most.append(tuple([k,v]))
    
most
#a = tuple(by_bnfname.nlargest(1)[0], by_bnfname.nlargest(1)[1])
#can also use: In [64]: res = g.apply(lambda x: x.order(ascending=False).head(3))
#by_bnfname.sort('items').head()

[('Omeprazole_Cap E/C 20mg', 218583)]

In [15]:
most_common_item = most

In [16]:
grader.score.dw__most_common_item(most_common_item)

Your score:  1.0


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

scripts.rename(columns = {'practice':'code'}, inplace = True)
merged_df = scripts.merge(practices, on=['code'])
#merged_df.head()
filtered = merged_df.groupby(['post_code', 'bnf_name'])['items']

filter2 = pd.DataFrame({'items' : merged_df.groupby( ['post_code', 'bnf_name'] ).size()}).reset_index()
filter3 = filter2.sort_values(by =['post_code', 'bnf_name'])
filter3

#def postalsum:
postallist = filter3['post_code'].unique()

print(scripts.head())
filter3
print(scripts.loc[scripts['bnf_name'].values == 'Bisacodyl_Tab E/C 5mg '])

In [17]:
asrt = practices.sort_values('post_code').drop_duplicates('code', keep = 'first')

amerge = scripts.merge(asrt, left_on = 'practice', right_on = 'code')

sumitemsbypost = amerge.groupby(['post_code','bnf_name'])[['items']].sum()
sumitemsbypost.reset_index(inplace=True)
#sumitemsbypost.head()
itemsmax = sumitemsbypost.groupby('post_code')[['items']].max()
#itemsmax.head()
#above code will find the value of item which is max
itemsmax.reset_index(inplace=True)
totalitems = itemsmax.merge(sumitemsbypost, on = ['post_code', 'items'], how= 'left')
#the above code is matching the bnfname corresponding max item
totalitems.head()



,post_code,items,bnf_name
0,B11 4BW,706,Salbutamol_Inha 100mcg (200 D) CFF
1,B12 9LP,425,Paracet_Tab 500mg
2,B18 7AL,556,Salbutamol_Inha 100mcg (200 D) CFF
3,B21 9RY,1033,Metformin HCl_Tab 500mg
4,B23 6DJ,599,Lansoprazole_Cap 30mg (E/C Gran)


In [18]:
# sorting and finding ratios
# below code will keep the first item in sorted bnf max table
sortedtotal = totalitems.sort_values('bnf_name').drop_duplicates(['post_code', 'items'], keep = 'first')
sortedtotal.head()

,post_code,items,bnf_name
73,E15 4ES,451,Amlodipine_Tab 10mg
66,DN16 2AB,1072,Amlodipine_Tab 5mg
255,WS9 8AJ,396,Amlodipine_Tab 5mg
142,NG2 7SD,605,Amlodipine_Tab 5mg
97,KT16 8HZ,401,Amlodipine_Tab 5mg


In [19]:
sortedtotal.sort_values('post_code', inplace = True)
sortedtotal.head()
# above code will again sort acc to postcode

postaltotal = sumitemsbypost.groupby('post_code')[['items']].sum().reset_index()
# this will find the sum of total items from a particular post

q3df = postaltotal.merge(sortedtotal, on ='post_code')
q3df.head()
#sortedtotal.shape
#postaltotal.shape
q3df['ratio'] = q3df['items_y'].astype(float)/q3df['items_x'].astype(float)
#q3df['ratio'].head()
filterdf= q3df[['post_code', 'bnf_name', 'ratio']]
filterdf = filterdf.head(100)
vlist = filterdf.values.tolist()
finallist = []
for item in vlist:
    finallist.append(tuple(item))
    





In [20]:
items_by_region = finallist

In [21]:
grader.score.dw__items_by_region(items_by_region)

Your score:  1.0


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [22]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

The Python join() method is a string method, and takes a list of things to join with the string. A simpler example might help explain:

>>> ",".join(["a", "b", "c"])
'a,b,c'

In [23]:
opioidslist = '|'.join(opioids)
opioidslist
chem.head()
nscript = scripts
opioidbnfcode = chem.loc[chem['NAME'].str.contains(opioidslist, case=False)]['CHEM SUB'].tolist()

nscript['opioid_prescription'] = nscript['bnf_code'].isin(opioidbnfcode)
nscript.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,opioid_prescription
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,False
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,False
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,False
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,False
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,False


In [24]:
practicecode = practices.loc[0]
practicecode
truevalues = nscript[nscript['opioid_prescription']==True]

#somevalue = nscript[nscript['practice']== 'N85639']


Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [25]:
prct = practices[['code', 'name']]
prct.columns = ['practice', 'name']
prct.head()

,practice,name
0,A81002,QUEENS PARK MEDICAL CENTRE
1,A81003,VICTORIA MEDICAL PRACTICE
2,A81004,WOODLANDS ROAD SURGERY
3,A81005,SPRINGWOOD SURGERY
4,A81006,TENNANT STREET MEDICAL PRACTICE


In [26]:
opioids_per_practice = nscript.groupby('practice')['opioid_prescription'].mean().rename('fraction')
opioids_per_practice.head()
finalmean = nscript['opioid_prescription'].mean()
finalstd = nscript['opioid_prescription'].std()
print(finalstd)
relative_opioids_per_practice = (opioids_per_practice - finalmean).rename('relative')
relative_opioids_per_practice
opioid = nscript.groupby('practice')['opioid_prescription'].sum().rename('opioid')
#opioid.head()

0.18579817605238425


How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

relative_opioids_per_practice = 

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [27]:
countpresc = nscript.groupby('practice')['bnf_code'].count().rename('countpresc')
countpresc.head()
standard_error_per_practice = (finalstd/(countpresc**0.5)).rename('standard error')
print(standard_error_per_practice.head())
opioid_scores = (relative_opioids_per_practice/standard_error_per_practice).rename('opioid_scores')
opioid_scores.head()

practice
A81005    0.004786
A81007    0.004873
A81011    0.004692
A81012    0.005091
A81017    0.004007
Name: standard error, dtype: float64


practice
A81005   -0.548306
A81007    1.544557
A81011    2.291795
A81012    1.373060
A81017    0.583168
Name: opioid_scores, dtype: float64

The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [28]:
combined = pd.concat([opioid, countpresc, opioids_per_practice, relative_opioids_per_practice, standard_error_per_practice,opioid_scores], axis=1)
combined = combined.reset_index()
combined.head()
prct.reset_index().head()
combinedfinal = combined.merge(prct, on='practice', how='left')


combinedfinal.sort_values('opioid_scores', ascending = False , inplace=True)
combinedfinal = combinedfinal.drop_duplicates('name')
combinedfinal.head()
result = combinedfinal[['name','opioid_scores','countpresc']]
result.head()

result = result.head(100)
v2list = result.values.tolist()
final3list = []
for item in v2list:
    final3list.append(tuple(item))

    


In [29]:
#unique_practices = 
anomalies = final3list
anomalies


[('NATIONAL ENHANCED SERVICE', 11.695817862936027, 7),
 ('OUTREACH SERVICE NH / RH', 7.339043019238823, 2),
 ('BRISDOC HEALTHCARE SERVICES OOH', 6.1505817490838295, 60),
 ('H&R P C SPECIAL SCHEME', 5.123032414033079, 36),
 ('HMR BARDOC OOH', 4.958866438487605, 321),
 ('INTEGRATED CARE 24 LTD (CWSX OOH)', 4.8888781604828235, 426),
 ('DARWEN HEALTHCARE', 4.8391589686363385, 1917),
 ('THE LIMES MEDICAL PRACTICE', 4.546841872334426, 1321),
 ('IC24 LTD (BRIGHTON & HOVE OOH)', 4.335047010605197, 357),
 ('OLDHAM 7 DAY ACCESS HUB2 OOH', 4.31178403661019, 56),
 ('IC24 LTD (NORFOLK & WISBECH OOH)', 4.2575005924727645, 489),
 ('ROSSENDALE MIU & OOH', 4.256827446322491, 18),
 ('BURY WALK-IN CENTRE', 4.150589122881536, 138),
 ('IC24 LTD (HORSHAM & MID SUSSEX OOH)', 3.7816207038443523, 215),
 ('LCW HOUNSLOW CCG OOH', 3.582848546206269, 69),
 ('WEEKEND WORKING EASINGTON NORTH', 3.565938771129764, 278),
 ('COMPASS ENFIELD', 3.5587202651067935, 7),
 ('BASSETLAW DRUG & ALCOHOL SERVICE', 3.53326410251154

In [30]:
results.head()

NameError: name 'results' is not defined

In [ ]:
grader.score.dw__script_anomalies(anomalies)

## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [31]:
scripts16 = pd.read_csv('./dw-data/201606scripts_sample.csv.gz')

In [32]:
growthrate = (scripts.bnf_name.value_counts() - scripts16.bnf_name.value_counts())/scripts16.bnf_name.value_counts()
resp = pd.DataFrame(dict(growthrate = growthrate, item16 = scripts16.bnf_name.value_counts())).reset_index()
resp.fillna(0, inplace=True)
resp = resp[resp['item16'] >= 50]
resp.sort_values(by='growthrate', ascending = False, inplace=True)
script_growth = list(pd.concat([resp.head(50), resp.tail(50)]).itertuples(index=False, name=None))


In [ ]:
grader.score.dw__script_growth(script_growth)

## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [33]:
unique_values = practices.sort_values(['code', 'post_code']).drop_duplicates(subset= 'code')
unique_values.head()
script_by_bnf_code = scripts.groupby(['bnf_code'])['bnf_code'].count().reset_index(name='bnf_code_count')
script_by_bnf_code['p'] = script_by_bnf_code['bnf_code_count'] / len(scripts)

In [34]:
p = 1 / len(script_by_bnf_code)
script_by_bnf_code['rare'] = np.where(script_by_bnf_code['p'] < 0.1 * p, True, False)
#rates = ...
#rare_codes = 
scripts_rare = pd.merge(scripts, script_by_bnf_code, how = 'inner', on = 'bnf_code')
scripts_rare.head()
#script_by_bnf_code.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,opioid_prescription,bnf_code_count,p,rare
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,False,1035,0.001064,False
1,N81013,0106020C0,Bisacodyl_Tab E/C 5mg,19,28.10,28.14,860,False,1035,0.001064,False
2,N81029,0106020C0,Bisacodyl_Tab E/C 5mg,50,67.73,67.51,2074,False,1035,0.001064,False
3,N81029,0106020C0,Bisacodyl_Suppos 10mg,1,1.77,1.75,6,False,1035,0.001064,False
4,N81029,0106020C0,Dulcolax_Tab 5mg,1,3.78,3.61,56,False,1035,0.001064,False


In [39]:
costofprac = scripts_rare.groupby('practice')['act_cost'].sum().reset_index(name = 'tpraccost')
rarecost = scripts_rare[scripts_rare['rare'] ==True].groupby('practice')['act_cost'].sum().reset_index(name = 'rare_cost')
script_cost_by_practice = pd.merge(costofprac, rarecost, on = 'practice', how='left')
script_cost_by_practice.fillna(0, inplace=True)
script_cost_by_practice['rare_cost_prop'] = (script_cost_by_practice['rare_cost'] / script_cost_by_practice['tpraccost'])
script_cost_by_practice.head()

,practice,tpraccost,rare_cost,rare_cost_prop
0,A81005,103840.82,1247.83,0.012017
1,A81007,113482.49,951.06,0.008381
2,A81011,159507.03,816.02,0.005116
3,A81012,83296.81,1145.11,0.013747
4,A81017,232656.17,1712.15,0.007359


In [36]:
total_cost = scripts_rare['act_cost'].sum()
total_rare_cost = scripts_rare[scripts_rare['rare'] == True]['act_cost'].sum()

overall_rare_cost = total_rare_cost / total_cost
script_cost_by_practice['relative_rare_cost_prop'] = script_cost_by_practice['rare_cost_prop'] - overall_rare_cost

In [37]:
script_cost_by_practice['standard_errors'] = script_cost_by_practice['relative_rare_cost_prop'].std()
script_cost_by_practice.head()

,practice,tpraccost,rare_cost,rare_cost_prop,relative_rare_cost_prop,standard_errors
0,A81005,103840.82,1247.83,0.012017,-0.003946,0.060509
1,A81007,113482.49,951.06,0.008381,-0.007582,0.060509
2,A81011,159507.03,816.02,0.005116,-0.010847,0.060509
3,A81012,83296.81,1145.11,0.013747,-0.002216,0.060509
4,A81017,232656.17,1712.15,0.007359,-0.008604,0.060509


In [38]:
script_cost_by_practice['z_scores'] = script_cost_by_practice['relative_rare_cost_prop'] / script_cost_by_practice['standard_errors']
print(script_cost_by_practice.head())
print(script_cost_by_practice.shape)
practices_unique = practices.sort_values('name').groupby('code', sort=False).first()
practices_unique.reset_index(inplace=True)
merge_script_practice_z_scores = script_cost_by_practice.merge(
    practices_unique[['name', 'code']], how = 'inner', left_on = 'practice', right_on = 'code', sort = False)

print(merge_script_practice_z_scores.shape)
merge_script_practice_z_scores = merge_script_practice_z_scores.sort_values('name', ascending = True)
merge_script_practice_z_scores.drop_duplicates(subset = ['code', 'name'], inplace = True)

rare_scripts = []
for index, row in merge_script_practice_z_scores.iterrows():
    rare_scripts.append(
        (row['practice'], 
         row['name'],
         row['z_scores']))

rare_scripts = sorted(rare_scripts, key=lambda x: x[2], reverse = True)[:100]

  practice  tpraccost  rare_cost  rare_cost_prop  relative_rare_cost_prop  \
0   A81005  103840.82    1247.83        0.012017                -0.003946   
1   A81007  113482.49     951.06        0.008381                -0.007582   
2   A81011  159507.03     816.02        0.005116                -0.010847   
3   A81012   83296.81    1145.11        0.013747                -0.002216   
4   A81017  232656.17    1712.15        0.007359                -0.008604   

   standard_errors  z_scores  
0         0.060509 -0.065216  
1         0.060509 -0.125308  
2         0.060509 -0.179263  
3         0.060509 -0.036615  
4         0.060509 -0.142190  
(856, 7)
(856, 9)


Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [ ]:
rare_cost_prop = ...

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [ ]:
relative_rare_cost_prop = ...

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

standard_errors = ...

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

rare_scores = ...

rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100

In [19]:
grader.score.dw__rare_scripts(rare_scripts)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*